# Hyperbolic neural net
> Do hyperbolic neural nets beat HDTs?

In [1]:
%load_ext autoreload
%autoreload 2

In [124]:
import torch
import argparse
import copy

from sklearn.model_selection import train_test_split

import sys

sys.path.append("/home/phil/americangut")
from make_xy import make_anndata_X_y

from hgcn.models.base_models import NCModel
from hgcn.models.encoders import HNN
from hgcn.optimizers.radam import RiemannianAdam


In [58]:
# Get data
X, y = make_anndata_X_y(balance=True, column="ibd", embedding="poi_mix_2")

# Convert to torch
X = torch.Tensor(X)
y = torch.Tensor(y)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)


In [77]:
# Initialize this kind of arcane HNN object:

C = 1.0  # c is the hyperbolic radius
EMBEDDING_DIM = 2  # dimension of the Poincare ball

hnn = HNN(
    c=C,
    args=argparse.Namespace(
        num_layers=2,
        manifold="PoincareBall",
        dropout=0.5,
        bias=True,
        act="tanh",
        feat_dim=EMBEDDING_DIM,
        dim=EMBEDDING_DIM,
        task="nc",
        c=C,
        cuda="gpu",
        device=torch.device("cuda:0" if torch.cuda.is_available() else "cpu"),
    ),
)


In [118]:
# Define a simple classifier on top of the HNN encoder (no adj matrix):


class HNN_classifier(torch.nn.Module):
    def __init__(self, c, embedding_dim, latent_dim):
        super(HNN_classifier, self).__init__()
        self.encoder = HNN(
            c=c,
            args=argparse.Namespace(
                num_layers=2,
                manifold="PoincareBall",
                dropout=0.5,
                bias=True,
                act="tanh",
                feat_dim=embedding_dim,
                dim=latent_dim,
                task="nc",
                c=C,
                cuda="gpu",
                device=torch.device(
                    "cuda:0" if torch.cuda.is_available() else "cpu"
                ),
            ),
        )
        self.classifier = torch.nn.Linear(latent_dim, 1, bias=True)
        self.sigmoid = torch.nn.Sigmoid()

    def forward(self, x):
        x = self.encoder.encode(x, None)
        x = self.classifier(x)
        x = self.sigmoid(x)
        return x


In [119]:
# Train! (need Riemannian Adam)

model = HNN_classifier(c=1, embedding_dim=2, latent_dim=4)
radam = RiemannianAdam(model.parameters(), lr=1)
losses = 0
for epoch in range(10000):
    radam.zero_grad()
    output = model(X_train).squeeze()
    loss = torch.nn.functional.binary_cross_entropy_with_logits(output, y_train)
    loss.backward()
    radam.step()
    losses += loss.item()
    if epoch % 1000 == 0:
        print(epoch, losses / 1000, sep="\t")
        losses = 0


0	0.000720930278301239
1000	0.6931653810739518
2000	0.6931470614671708
3000	0.6931576954126358
4000	0.6931473612785339
5000	0.6931473612785339
6000	0.6931473612785339
7000	0.6931473612785339
8000	0.6931473612785339
9000	0.6931473612785339


In [132]:
# Ok, this might have been wrong. Let's try to use NCmodel:

# copy argparse namespace
args_nc = argparse.Namespace(
    num_layers=2,
    manifold="PoincareBall",
    dropout=0.5,
    bias=True,
    act="tanh",
    feat_dim=EMBEDDING_DIM,
    dim=EMBEDDING_DIM,
    task="nc",
    c=C,
    cuda="gpu",
    device=torch.device("cuda:0" if torch.cuda.is_available() else "cpu"),
    n_nodes=len(X_train),
    n_classes=2,
    model="HNN",
    pos_weight=False
)

classifier = NCModel(args_nc)


In [134]:
classifier.train()

NCModel(
  (encoder): HNN(
    (layers): Sequential(
      (0): HNNLayer(
        (linear): HypLinear(in_features=2, out_features=2, c=tensor([1.], device='cuda:0'))
        (hyp_act): HypAct(c_in=tensor([1.], device='cuda:0'), c_out=tensor([1.], device='cuda:0'))
      )
    )
  )
  (decoder): LinearDecoder(
    in_features=2, out_features=2, bias=True, c=tensor([1.], device='cuda:0')
    (cls): Linear(
      (linear): Linear(in_features=2, out_features=2, bias=True)
    )
  )
)